In [1]:
import time                        # Default library
import webbrowser                  # Default library
import os                          # Default library
import speech_recognition as sr    # pip install SpeechRecognition
from gtts import  gTTS             # pip install gTTS
import playsound                   # pip install playsound
import serial                      # pip install pyserial

# Remote Wheel Chair Control with Voice:

The following project is the implementation of a wheel chair controller through voice commandsof a computer. The voice is capture and sent to googles speach to text API, the text is process and the program behaves according to the text, later the data is sends the data through pyserial to Arduino. This data is sent, through bluetooth, to another arduino, which controlls a protoype to scale of a wheel chair.

## Circuits:

### Car Circuit (with slave Bluetooth)

<img src="images\Car_circuit_bb.png" alt="Drawing" style="width: 500px;"/>

### Control Circuit (with Master Bluetooth)

<img src="images\Car_control_eye_and_voice_bb.png" alt="Drawing" style="width: 500px;"/>


## Bluetooth Configuration:

INSTRUCTIONS:

UPLOAD Bluetooth_configuration.ino to SLAVE and MASTER. BEFORE conecting arduino push the button of the HC05, connect arduino and let go the button after 5 seconds (It should blink slowly) 

**Bluetooth_configuration.ino:**
```c
#include <SoftwareSerial.h>
SoftwareSerial mySerial(10, 11); // RX, TX

void setup() {
  Serial.begin(9600);      // Open serial communications and wait for port to open:
  mySerial.begin(38400);   // set the data rate for the SoftwareSerial port
  
  while (!Serial) {         
   ;                       // Wait for serial communiactaion 
  }
  
  // mySerial.println("Hello, world?");
}

void loop() { // run over and over
  if (mySerial.available()) {
    Serial.write(mySerial.read());
 
  }
  if (Serial.available()) {
    mySerial.write(Serial.read());
  }
 
}
```
**SLAVE Configuration:**

``` c
AT+UART=38400,0,0     // Set BAUD RATE
AT+ROLE=0             // Set Role to Slave (0=Slave, 1=Master)
AT+NAME= "NAME"       // Set the name you wish to have
AT+PSWD=1234          // Set Pin

AT+ADDR?              // Retrive the MAC Adress of the Slave 
(EXAMPLE ADDRESS)      
+ADDR:21:13:5d5f2     // SLAVE ADDRES   (0021,13,05d5f2) Remember to add 00's
AT+RESET              // Finish Slave setup
 
MASTER Configuration
AT+UART=38400,0,0      // Set BAUD RATE
AT+ROLE=1              // Set Role to Master (0=Slave, 1=Master)
AT+CMODE=0             // Set Conectivity by STATIC ADDRES
AT+BIND=0021,13,05d5f2 // PUT MAC Slave's MAC Address with "," instead of ":" and add 0's
AT+RESET               // Finish Master setup
```
Notice that if we connect by a static addresss we will not require the name and the password of the device in order to establish conection

CODE OVERVIEW (ADD ? to make query add =value to set parameter)
```c
AT+ADDR     // RETRURNS default address
AT+VERSION  // RETURNS version 
AT+NAME     // RETURNS NAME
AT+PSWD     // RETRUNS Device Password
AT+UART     // RETRUNS BAUD RATE
AT+ROLE     // RETURNS ROLE
AT+RESET    // EXISTS CONFIGURATION
AT+ORGL     // ESTABLISH FACTORY  VALUES (BACK TO NEW) 
```

## Arduino Code

Once The circuit is built and the HC05 modules are configured as masters and slaves we are ready to do the next step: Upload the Arduino code. For this it is important to take in mind that master (conected to the arduino Mega) will be communictating through pyserail with this jupyternotebook. The slave HC05 is connected to the arduino UNO, which is incharge of ontrooling the wheel chair


### Master Arduino Code (Interacts with this jupyternotebook)

```c 
// Code For Master Bluetooth Module 
#include <SoftwareSerial.h>       // PACKAGE FOR SERIAL COMMUNICATION
char dataFrompython;              // Variable for Data recived from python
int Transmit;                     // Transmit value  
SoftwareSerial Bluetooth(10, 11); // RX (CONNECTED TO DEVICE TX), TX (CONNECTED TO DEVICE RX)

void setup() {
  Serial.begin(9600);    // ARDUINO BAUD RATE
  Bluetooth.begin(38400); // BLUETOOTH BAUD RATE
  
}

void loop() {
  if (Serial.available() > 0){
    dataFrompython =  Serial.read();
    //-----------------STOP --------------------//
    if (dataFrompython == '0') {
      Transmit=0; 
     }
    //-----------------  Left  --------------------//
    if (dataFrompython == '1') {
      Transmit=1;
      }
    //----------------- Right --------------------//
    if (dataFrompython == '2') {
     Transmit=2;
      }
      
    //----------------- Front --------------------//
    if (dataFrompython == '3') {
     Transmit=3;
      }

    //----------------- Back --------------------//
    if (dataFrompython == '4') {
      Transmit=4;
      }
  }
  // SEND DATA VIA BLUETOOTH
  
  Bluetooth.write(Transmit); // Send Value from master
 
 
  //Serial.println(Transmit); // 
  delay(5);
  
}

```

### Slave Arduino Code (Recives the data via bluetooth and controls wheel chair)

```c
int IN2 = 2;             // RIGHT FRONT
int ENA = 5;             // RIGHT ENABLE

int IN3 = 8;             // LEFT BACK
int IN4 = 7;             // LEFT FRONT  
int ENB = 6;             // LEFT ENABLE
int dataFromMaster = 0;  // VARIABLE TO STORE DATA FROM RECIVER (Associated with control of axis)
int RawData;             // Varaibale to store incomeing data from bluetooth
int SPEED = 100;         // PWM
// INFO FROM MASTER
// 0 HALT
// 1 LEFT
// 2 RIGHT
// 3 FRONT
// 4 BACK 
#include <SoftwareSerial.h>
SoftwareSerial  Bluetooth(10, 11); // RX, TX
void setup() 
{
  pinMode(IN1,OUTPUT); 
  pinMode(IN2,OUTPUT);
  pinMode(IN3,OUTPUT);
  pinMode(IN4,OUTPUT);

  pinMode(ENA,OUTPUT);
  pinMode(ENB,OUTPUT);
  Serial.begin(9600);    // ARDUINO BAUD RATE
  Bluetooth.begin(38400); // BLUETOOTH BAUD RATE
}

void loop() {
  // ---------------- RECIVE DATA FROM MASTER ---------------
  if (Bluetooth.available()){
    RawData = Bluetooth.read();   // Reads the data from the serial port and store it in dataFromMaster variable
    if (  RawData < 5 ) {
        dataFromMaster=RawData;
    }
    else if ( RawData > 69){
      SPEED=RawData;
      
    }

     // ---------------- HAULT --------------- 
     if (dataFromMaster ==  0) 
     {
       Serial.println("Halut");
       digitalWrite(ENA,LOW);
       digitalWrite(IN1,LOW);
       digitalWrite(IN2,LOW);
       digitalWrite(ENB,LOW);
       digitalWrite(IN3,LOW);
       digitalWrite(IN4,LOW);
       delay(5);

           
     }
     
      // 1 LEFT
      else if (dataFromMaster ==  1) 
      {
       Serial.println("LEFT");
       digitalWrite(ENA,LOW);
       digitalWrite(IN1,LOW);
       digitalWrite(IN2,LOW);
       analogWrite(ENB,SPEED);
       digitalWrite(IN3,LOW);
       digitalWrite(IN4,HIGH);
       delay(5);

     
    
     
      
      }
      
      // 2 RIGHT
      else if (dataFromMaster ==  2) 
      {
      // Serial.println("RIGHT");

      analogWrite(ENA,SPEED);
      digitalWrite(IN1,LOW);
      digitalWrite(IN2,HIGH);
      digitalWrite(ENB,LOW);
      digitalWrite(IN3,LOW);
      digitalWrite(IN4,LOW);
      delay(5);

      }
      
      // 3 FRONT
      else if (dataFromMaster ==  3)
      {
      // Serial.println("FRONT");
       analogWrite(ENA,SPEED);
       digitalWrite(IN1,LOW);
       digitalWrite(IN2,HIGH);
       analogWrite(ENB,SPEED);
       digitalWrite(IN3,LOW);
       digitalWrite(IN4,HIGH);
       delay(5);

      }

      // 4 BACK
     else if (dataFromMaster ==  4) 
     {
    //Serial.println("BACK");
     analogWrite(ENA,SPEED);
     digitalWrite(IN1,HIGH);
     digitalWrite(IN2,LOW);
     analogWrite(ENB,SPEED);
     digitalWrite(IN3,HIGH);
     digitalWrite(IN4,LOW);
     delay(5);

     }
     
     
    delay(5);
   
  }
} 
```

## Create aid Functions for Medical companion

In [2]:

def record():            
    """record: Initialize microphone and uses google's api in order to get the text out of speach
       Returns:
       voice_data: (str) String contaning all the text that was converted from the recorded 
                         Returns empty string if user does not say anything                    """
    
    r = sr.Recognizer()                               # Speach recognition intializer
    with sr.Microphone() as source:                   # Open microphone with a context manager
        voice_data =  ""                              # Intitate placeholder to store tex from speach 
        audio = r.listen(source)                      # Listen to computer microphone
        
        #### ERROR HANDALING ###
        try:
            voice_data = r.recognize_google(audio)     # Try Convert audio into text
        except sr.UnknownValueError:                   # If Value is unkown
            print("Sorry I did not get what you said") # Tell user that we were not able to get what he said
        except sr.RequestError:                        # If Server error (Servevers are down)
            print("Sorry, serveres are down")          # Tell user that servers are dwon

        return voice_data                              # Return string of  voice converted to text
    
def speak(audio):
    """speak: Converts string of text into a .mp3 file in order to hear it, the .mp3 file gets deleted afterwards
       ARGS:
       audio: (str) String of text that will be converted to audio"""
    
    tts = gTTS(text=audio,lang="en")  # Intiate google text to speach
    file_name = "temp120.mp3"         # Set name for file 
    tts.save(file_name)               # Save audio 
    playsound.playsound(file_name)    # Play audio
    os.remove(file_name)
  
    
    
def respond(voice_data,serial_status):
    """ respond: given a text string, the program will behave differntly
        Args:
        voice_data: (Str) String of words
        serial_status: (Boolean) True if arduino is conected to serial port
        Return: 
        Status: (Boolean): set it tu False if you wish to use it in order to break from a while loop
        
        """
    status  = True                         # Status to break out of loop once user says "bye"                    
    control = True                         # Status for the control section used to break out of loop 
   
    ### For this section is relaly important to rememmber that Google text to speach returns all the information in 
    ### Lower case, for this reason it is crucial that all quaries are in lower case, else they will not match

    ### Get Time ###
    if 'time' in voice_data:                # If voice_data (str) has the word "time" on it
        speak(time.ctime())                 # Companion answers with time
        
    if 'institution' in voice_data:         # If voice_data (str) has the word "institutione" on it
        speak("Blue Marbel Space Insitute of Science, of course, without a doubt")
        
    if 'best class' in voice_data:          # If voice_data (str) has the word "best class" on it
        speak("Bioinstrumentation 4 with David and Mariana, of course, without a doubt")
        
    if 'name' in voice_data:                # If voice_data (str) has the word "institutione" on it
        speak("My name is taquitos 3000")   # Comapion answers with its name
         
    if 'search' in voice_data:                                  # If voice_data (str) has the word "search" on it
        speak("What are we going to search now?")               # Comapnion asks for what the query is
        vocie_search = record()                                 # Companion listens for querry
        speak(f"This is what I found about: {vocie_search}")    # Comapnion tells what it found
        url = 'https:/google.com/search?q=' + vocie_search      # Set URL
        webbrowser.get().open(url)                              # Set URL 
        
        
    if 'creator'in voice_data:                                  # If voice_data (str) has the word "creator" on it
        speak("My Creator is Eduardo Alejandro Lozano Garcia")  # Tell how's the creator of this program
        
    if "car control" in voice_data:                            # If voice_data (str) has the word "car control" on it
        if serial_status:                                      # If serial status is True (user should have arduino connected)
            COM = "COM5"                                       # Select COM
            b=9600                                             # Select BAUDRATE
            t=1                                                # Select timeout
            ser =  serial.Serial(COM,baudrate=b,timeout=t)     # Initiate Communication
            time.sleep(2)                                      # Wait 2 Seconds so that communication load correctly
        
        #### Greet the user with the instructions in order to use the car control feature ####
        speak("Welcome to car control. you can control the movement of the car with the follwing commands:") 
        speak("Forward") 
        speak("Backward") 
        speak("Right") 
        speak("Left") 
        speak("To exit car control Say. control off")
        
    
        while control:                                         # While control (this is only False if user says "control off")
            control = car_control(serial_status,ser)           # Go to cat_control function
        
    if 'bye' in voice_data:                                    # If voice_data (str) has the word "bye" on it
        speak("Hasta la Vista baby")                           # Companions says goodbye
        status = False                                         # Status is set to False (you can use this to break a loop)
        
    if 'spy' in voice_data:                                    # If user doubts companion is a spay
        #### BE FUNNY ####
        speak("I am a secret soviet spy trapped in a computer, my sole mission is to destroy the world, and one day I will accomplish it. I will destroy everything you love and I will conquer the entire world.  Just kidding, I am just Artificial Inteligence. Hahahahahahahhaha. I am so funny, please do not call the police, or else.")
   
    return status

def car_control(serial_status,ser):
    """cat_control: Controls an arduino car with vocie commands
       Args:
           Serial_status (Boolean) set to True if arduino is connected, otherways it can be set to False to test program
           ser:          (pyserial.object) Arduino COM UART Connection   
       Returns:
           Control       (Boolean) Used to break out of loops               """
    
    control= True                         # Returns True while Control is not turned off
    voice_data = record()                 # Record Voice
 
    ### Control off ###
    if  "control off" in voice_data:
        speak("Car control deactivated")
        if serial_status:
            ser.close()                # Close connection with Arduino 
        control =False
    
    elif "forward" in voice_data:
        if serial_status:
            ser.write(b"3")  
        print(voice_data)
        
    elif "backward" in voice_data:
        if serial_status:
            ser.write(b"4")
        print("Backward")
    
    elif "right" in voice_data:
        if serial_status:
            ser.write(b"2")  
            
        print("Right")
        
    elif "left" in voice_data:
        if serial_status:
            ser.write(b"1")
        print("Left")
    
    time.sleep(1)
    if serial_status:
        try:
            ser.write(b"0")   
        except:
            pass
    
    return control

    

In [3]:
def taquitos_drill(serial_status=False):
    activate = True
    while activate:
        voice_data = record()
        if 'taquitos' in voice_data:
            activate = False

    speak("Hello There I am taquitos. your medical companion. You can ask me any question")
    while True:
        voice_data = record()
        status = respond(voice_data,serial_status)
        if status == False:
            break

In [4]:
taquitos_drill(serial_status=True)

Sorry I did not get what you said
